In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
test_df_backup = test_df.copy()

In [3]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#The features Paseesnger ID, Name and ticket are not major factors for survival
#so lets drop them
train_df = train_df.drop(["PassengerId"], axis = 1)
train_df = train_df.drop(["Name"], axis = 1)
train_df = train_df.drop(["Ticket"], axis = 1)

In [6]:
train_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [7]:
#too many values (77%) are missing for the column, so lets drop it
train_df = train_df.drop(["Cabin"], axis = 1)

#Scope for improvement 1: now age is filled with just the mean, instead,
#for age of passenger lets fill in ages with random numbers within one standard deviation of mean
#as this region includes 68% of the values and also doesnt put in small / large values
age_mean = train_df["Age"].mean()
train_df["Age"].fillna(age_mean, inplace = True)

#lets fill in embarked with 'S' as most people embarked in Southampton
train_df["Embarked"].fillna('S', inplace = True)


In [8]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [9]:
# converting data into numeric

#Sex
Sex_dict = {"male":1, "female":0}
train_df["Sex"] = train_df["Sex"].map(Sex_dict)

#Embarked
train_df["Embarked_C"] = (train_df["Embarked"] == 'C')
train_df["Embarked_Q"] = (train_df["Embarked"] == 'Q')
train_df["Embarked_S"] = (train_df["Embarked"] == 'S')
train_df = train_df.drop(["Embarked"], axis = 1)

In [10]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,False,False,True
1,1,1,0,38.0,1,0,71.2833,True,False,False
2,1,3,0,26.0,0,0,7.9250,False,False,True
3,1,1,0,35.0,1,0,53.1000,False,False,True
4,0,3,1,35.0,0,0,8.0500,False,False,True


In [11]:
#lets convert the age into a more useful form

train_df["Child"] = (train_df["Age"] <= 11)
train_df["Young"] = (train_df["Age"] > 11) & (train_df["Age"] <= 20)
train_df["Adult"] = (train_df["Age"] > 20) & (train_df["Age"] <= 35)
train_df["Mature"] = (train_df["Age"] > 35) & (train_df["Age"] <= 50)
train_df["Old"] = (train_df["Age"] > 50) 

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Child,Young,Adult,Mature,Old
0,0,3,1,22.0,1,0,7.2500,False,False,True,False,False,True,False,False
1,1,1,0,38.0,1,0,71.2833,True,False,False,False,False,False,True,False
2,1,3,0,26.0,0,0,7.9250,False,False,True,False,False,True,False,False
3,1,1,0,35.0,1,0,53.1000,False,False,True,False,False,True,False,False
4,0,3,1,35.0,0,0,8.0500,False,False,True,False,False,True,False,False


In [12]:
#normalize the Fare and Age and store the skewing values:

age_mean = train_df["Age"].mean()
age_stddev = train_df["Age"].std()
train_df["Age"] = (train_df["Age"] - age_mean)/age_stddev

fare_mean = train_df["Fare"].mean()
fare_stddev = train_df["Fare"].std()
train_df["Fare"] = (train_df["Fare"] - fare_mean)/fare_stddev

In [13]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Child,Young,Adult,Mature,Old
0,0,3,1,-0.592148,1,0,-0.502163,False,False,True,False,False,True,False,False
1,1,1,0,0.638430,1,0,0.786404,True,False,False,False,False,False,True,False
2,1,3,0,-0.284503,0,0,-0.488580,False,False,True,False,False,True,False,False
3,1,1,0,0.407697,1,0,0.420494,False,False,True,False,False,True,False,False
4,0,3,1,0.407697,0,0,-0.486064,False,False,True,False,False,True,False,False


In [14]:
#do the same processes to the test dataframe

#drop the useless
test_df = test_df.drop(["PassengerId"], axis = 1)
test_df = test_df.drop(["Name"], axis = 1)
test_df = test_df.drop(["Ticket"], axis = 1)

#take care of nan
test_df = test_df.drop(["Cabin"], axis = 1)
test_df["Age"].fillna(age_mean, inplace = True)
test_df["Embarked"].fillna('S', inplace = True)
#test df has one value of fare nan
test_df["Fare"].fillna(test_df["Fare"].mean(), inplace = True)

#convert to numeric
#Sex
test_df["Sex"] = test_df["Sex"].map(Sex_dict)
#Embarked
test_df["Embarked_C"] = (test_df["Embarked"] == 'C')
test_df["Embarked_Q"] = (test_df["Embarked"] == 'Q')
test_df["Embarked_S"] = (test_df["Embarked"] == 'S')
test_df = test_df.drop(["Embarked"], axis = 1)

#convert age
test_df["Child"] = (test_df["Age"] <= 11)
test_df["Young"] = (test_df["Age"] > 11) & (test_df["Age"] <= 20)
test_df["Adult"] = (test_df["Age"] > 20) & (test_df["Age"] <= 35)
test_df["Mature"] = (test_df["Age"] > 35) & (test_df["Age"] <= 50)
test_df["Old"] = (test_df["Age"] > 50) 

#Normalize Age and fare using the same skewing constants
test_df["Age"] = (test_df["Age"] - age_mean)/age_stddev
test_df["Fare"] = (test_df["Fare"] - fare_mean)/fare_stddev

test_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Child,Young,Adult,Mature,Old
0,3,1,0.369241,0,0,-0.490508,False,True,False,False,False,True,False,False
1,3,0,1.330631,1,0,-0.507194,False,False,True,False,False,False,True,False
2,2,1,2.484298,0,0,-0.453112,False,True,False,False,False,False,False,True
3,3,1,-0.207592,0,0,-0.473739,False,False,True,False,False,True,False,False
4,3,0,-0.592148,1,1,-0.400792,False,False,True,False,False,True,False,False


In [15]:
X = train_df.drop(["Survived"], axis = 1).values.astype('float32')
y = train_df["Survived"].values.astype('int32')

X_test = test_df.values.astype('float32')

In [16]:
from sklearn.model_selection import train_test_split

In [58]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size = 0.30)

In [59]:
X_train

array([[ 3.0000000e+00,  0.0000000e+00, -5.9214801e-01, ...,
         1.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 3.0000000e+00,  1.0000000e+00,  4.3718928e-15, ...,
         1.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 3.0000000e+00,  0.0000000e+00, -2.1303711e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 1.0000000e+00,  1.0000000e+00,  1.6382755e+00, ...,
         0.0000000e+00,  0.0000000e+00,  1.0000000e+00],
       [ 3.0000000e+00,  1.0000000e+00,  4.3718928e-15, ...,
         1.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 3.0000000e+00,  1.0000000e+00,  4.3718928e-15, ...,
         1.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [60]:
import tensorflow as tf

In [203]:
#initialize model

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(25, activation = 'relu', input_shape=(14,)),
    tf.keras.layers.Dropout(0.22),
    tf.keras.layers.Dense(5, activation = 'relu'),
    tf.keras.layers.Dense(1)
])


#predictions
y_pred = model(X_train[:1]).numpy()
y_pred

array([[-0.45211443]], dtype=float32)

In [204]:
adam_op = tf.keras.optimizers.Adam(
    learning_rate= 0.001 , beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')


model.compile(optimizer = adam_op,
             loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics = ['accuracy'])

In [206]:
model.fit(X_train, y_train, epochs = 25)

Train on 623 samples
Epoch 1/25
623/623 [==============================] - 0s 55us/sample - loss: 0.4452 - accuracy: 0.7833
Epoch 2/25
623/623 [==============================] - 0s 49us/sample - loss: 0.4253 - accuracy: 0.8042
Epoch 3/25
623/623 [==============================] - 0s 50us/sample - loss: 0.4138 - accuracy: 0.8299
Epoch 4/25
623/623 [==============================] - 0s 52us/sample - loss: 0.4159 - accuracy: 0.7994
Epoch 5/25
623/623 [==============================] - 0s 50us/sample - loss: 0.4106 - accuracy: 0.8138
Epoch 6/25
623/623 [==============================] - 0s 53us/sample - loss: 0.4121 - accuracy: 0.8299
Epoch 7/25
623/623 [==============================] - 0s 52us/sample - loss: 0.4111 - accuracy: 0.8074
Epoch 8/25
623/623 [==============================] - 0s 51us/sample - loss: 0.4219 - accuracy: 0.8202
Epoch 9/25
623/623 [==============================] - 0s 53us/sample - loss: 0.4104 - accuracy: 0.8202
Epoch 10/25
623/623 [==============================]

In [207]:
dev_loss, dev_accuracy = model.evaluate(X_dev, y_dev, verbose = 2)

268/268 - 0s - loss: 0.3964 - accuracy: 0.8134


In [166]:
# iteration over different NN sizes:

# size                train_loss    train_accuracy    dev_loss   dev_accuracy  dropout           inference
# 15 10 10 10 1 ..... 0.4592        0.8069            0.4386     0.7982        no overfitting    
# 15 10 1 ........... 0.4171        0.8204            0.4153     0.8161        no overfitting     
# 7 5 5 1 ........... 0.3919        0.8293            0.4239     0.8341        no overfitting     
# 20 5 1 ............ 0.4198        0.8278            0.4271     0.8072        no overfitting     
# 25 5 1 ............ 0.3811        0.8398            0.4522     0.8430        0.1 0             25 best for first layer    
# 30 5 1 ............ 0.3948        0.8308            0.4024     0.8341        0.1 0
# 25 5 5 1 .......... 0.3657        0.8473            0.4108     0.8430        0.1 0            

In [167]:
predictions = model(X_test).numpy()
predictions = 1/(1 + np.exp(-predictions)) #Applying sigmiod
binary_predictions = predictions > 0.5
print(predictions[1:10])

[[0.49504286]
 [0.05975993]
 [0.10755281]
 [0.6846236 ]
 [0.15865888]
 [0.67630464]
 [0.19520847]
 [0.7040332 ]
 [0.09140281]]


In [168]:
print(np.shape(binary_predictions))

(418, 1)


In [169]:
submission = pd.DataFrame(test_df_backup["PassengerId"])
submission["Survived"] = binary_predictions.astype("int32")
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [170]:
submission.to_csv('submissions.csv', index = False)